# Résolution d'une grille de sudoku

ex: https://fr.wikipedia.org/wiki/Math%C3%A9matiques_du_sudoku#/media/Fichier:Sudoku00.png

Notebook contenant mon étude réalisée sur les Sudokus:
1. Résolution d'une grille de sudoku
2. Vérification de l'unicité de la solution
3. Génération de sudoku, à difficultées variables

In [1]:
import numpy as np
import random as rd

# Résolution d'une grille de Sudoku

- L'ensemble de fonctions ci-dessous permet la résolution d'une grille de sudoku
- La fonction pos_verif (et most_val) définies les contraintes liées au Sudoku
- La fonction rempl_grille remplie de manière récursive la grille, qui est modifiée directement dans la variable d'entrée

In [2]:
def pos_verif(grille,x,y):
    if not most_val(grille[x,:]):
        return False
    if not most_val(grille[:,y]):
        return False
    
    deb_x = x//3
    deb_y = y//3
    bloc = np.array([0,0,0,0,0,0,0,0,0])
    for i in range(0,3):
        for j in range(0,3):
            bloc[3*i + j] = grille[3*deb_x + i,3*deb_y + j]
            
    if not most_val(bloc):
        return False
    return True

def most_val(liste):
    for i in range(1,10):
        count = 0
        for j in liste:
            if j==i:
                count+=1
        if count>1:
            return False
    return True

def rempl_grille(grille,pos):
    if pos==81:
        return True
    
    x = pos//9
    y = pos%9
    
    if grille[x][y] !=0:
        return rempl_grille(grille,pos+1)
    
    liste = list(range(1,10))
    rd.shuffle(liste)
    for n in range(0,9):
        grille[x][y] = liste[n]
        
        if pos_verif(grille,x,y):
            if rempl_grille(grille,pos+1):
                return True
        if n+1==9:
            grille[x][y]=0
            return False


In [3]:
%%time
grille = np.array([[1,0,0,0,0,0,0,0,6],
                   [0,0,6,0,2,0,7,0,0],
                   [7,8,9,4,5,0,1,0,3],
                   [0,0,0,8,0,7,0,0,4],
                   [0,0,0,0,3,0,0,0,0],
                   [0,9,0,0,0,4,2,0,1],
                   [3,1,2,9,7,0,0,4,0],
                   [0,4,0,0,1,2,0,7,8],
                   [9,0,8,0,0,0,0,0,0]])

rempl_grille(grille,0)

Wall time: 103 ms


True

In [4]:
grille

array([[1, 2, 3, 7, 8, 9, 4, 5, 6],
       [4, 5, 6, 1, 2, 3, 7, 8, 9],
       [7, 8, 9, 4, 5, 6, 1, 2, 3],
       [2, 3, 1, 8, 9, 7, 5, 6, 4],
       [5, 6, 4, 2, 3, 1, 8, 9, 7],
       [8, 9, 7, 5, 6, 4, 2, 3, 1],
       [3, 1, 2, 9, 7, 8, 6, 4, 5],
       [6, 4, 5, 3, 1, 2, 9, 7, 8],
       [9, 7, 8, 6, 4, 5, 3, 1, 2]])

# Unicité de la solution
Les fonctions ci-dessous permettent de vérifier l'unicité de la solution d'une grille de Sudoku. La vérification est empirique: si 10 résolutions sont égales, la solution est considérée comme unique

In [5]:
def equal(grid1,grid2):
    for i in range(0,9):
        for j in range(0,9):
            if grid1[i,j] != grid2[i,j]:
                return False
    return True

def unique_solution(grille):
    grid = [grille.copy()]
    rempl_grille(grid[0],0)
    
    for i in range(1,10):
        grid.append(grille.copy())
        rempl_grille(grid[i],0)
        if not equal(grid[i],grid[0]):
            return False
    return True

In [6]:
grille = np.array([[1,0,0,0,0,0,0,0,6],
                   [0,0,6,0,2,0,7,0,0],
                   [7,8,9,4,5,0,1,0,3],
                   [0,0,0,8,0,7,0,0,4],
                   [0,0,0,0,3,0,0,0,0],
                   [0,9,0,0,0,4,2,0,1],
                   [3,1,2,9,7,0,0,4,0],
                   [0,4,0,0,1,2,0,7,8],
                   [9,0,8,0,0,0,0,0,0]])
unique_solution(grille)

True

# Génération de grille de sudoku à difficultées variables

In [7]:
def create_sudoku(difficulty = 'hard'):
    diff = {'easy':20,'medium':30,'hard':40}
    grille = np.zeros(shape = (9,9),dtype = int)
    rempl_grille(grille,0)
    pos = list(range(0,81))
    rd.shuffle(pos)
    
    for n in range(0,10):
        num = pos.pop()
        x = num//9
        y = num%9
        grille[x,y] = 0
    
    if not unique_solution(grille):
        return create_sudoku(difficulty)
    else:
        for n in range(0,diff[difficulty]):
            print('{}/{}, size(pos) = {}'.format(n,diff[difficulty],len(pos)),end = '\r')
            while True:
                if len(pos)==0:
                    return grille
                
                m = rd.randint(0,len(pos)-1)
                num = pos.pop(m)
                
                x = num//9
                y = num%9
    
                reminder = grille[x,y]
                grille[x,y] = 0
            
                if not unique_solution(grille):
                    grille[x,y] = reminder
                else:
                    break
    return grille

In [11]:
grille = create_sudoku(difficulty = 'easy')

In [ ]:
grille,unique_solution(grille)

(array([[0, 0, 9, 2, 4, 6, 5, 0, 1],
        [7, 0, 5, 1, 8, 0, 4, 9, 2],
        [1, 4, 0, 0, 0, 9, 0, 6, 8],
        [5, 7, 1, 8, 2, 4, 9, 3, 0],
        [0, 2, 3, 0, 9, 1, 7, 0, 0],
        [0, 9, 8, 3, 0, 0, 0, 1, 0],
        [0, 0, 0, 9, 0, 0, 1, 4, 0],
        [3, 0, 7, 0, 1, 8, 0, 2, 9],
        [0, 0, 4, 6, 3, 2, 8, 5, 7]]), True)